In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import ast

In [2]:
#declaracoes = pd.read_csv("declaracoes-nlp-new.csv", converters={'entities': lambda x: x[1:-1].split(',')})
#declaracoes

In [3]:
declaracoes_nopunct = pd.read_csv("declaracoes-nlp-no-punct.csv", converters={'tokens': lambda x: x[1:-1].split(','), 'entities': lambda x: x[1:-1].split(',')})
declaracoes_nopunct

,speaker,partido,speech,filename,no_punct,tokens,lemma,entities
0,Emília Cerqueira,PSD,"Sr. Presidente, Srs. Deputados, saúdo Os Verde...",darl13sl03n050.txt,sr presidente srs deputados saúdo os verdes po...,"['sr', 'presidente', 'srs', 'deputados', '...","['sr', 'presidente', 'sr', 'deputar', 'saudar'...","[(sr presidente srs, 'PER', 4317129024397789..."
1,Emília Cerqueira,PSD,"Portanto, estamos a falar, infelizmente, de um...",darl13sl03n050.txt,portanto estamos a falar infelizmente de uma s...,"['portanto', 'estamos', 'a', 'falar', 'inf...","['portanto', 'estar', 'o', 'falar', 'infelizme...","[(espanha, 'LOC', 385)]"
2,Emília Cerqueira,PSD,"Terminarei, Sr. Presidente, perguntando a Os V...",darl13sl03n050.txt,terminarei sr presidente perguntando a os verd...,"['terminarei', 'sr', 'presidente', 'pergunt...","['terminar', 'sr', 'presidente', 'perguntar', ...","[(terminarei sr, 'PER', 4317129024397789502)..."
3,Emília Cerqueira,PSD,Não é verdade!,darl13sl03n050.txt,não é verdade,"['não', 'é', 'verdade']","['não', 'ser', 'verdade']",[]
4,Emília Cerqueira,PSD,"Sr. Presidente, Sr. Deputado Pedro Soares, hoj...",darl13sl03n050.txt,sr presidente sr deputado pedro soares hoje te...,"['sr', 'presidente', 'sr', 'deputado', 'pe...","['sr', 'presidente', 'sr', 'deputar', 'pedrar'...","[(sr presidente sr deputado pedro soares, 'PE..."
...,...,...,...,...,...,...,...,...
92570,Heloísa Apolónia,PEV,"Sr. Presidente, Sr.as e Srs. Deputados: O prob...",darl13sl03n058.txt,sr presidente sras e srs deputados o problema ...,"['sr', 'presidente', 'sras', 'e', 'srs', ...","['sr', 'presidente', 'sr', 'e', 'sr', 'deputar...","[(sr presidente sras, 'PER', 431712902439778..."
92571,Heloísa Apolónia,PEV,"É por estas razões e por tantas outras, Sr. Pr...",darl13sl03n058.txt,é por estas razões e por tantas outras sr pres...,"['é', 'por', 'estas', 'razões', 'e', 'por...","['ser', 'por', 'este', 'razão', 'e', 'por', 't...",[]
92572,António Filipe,PCP,"Sr. Presidente, Sr.ª Ministra, Sr.ª Secretária...",darl13sl03n058.txt,sr presidente srª ministra srª secretária de e...,"['sr', 'presidente', 'srª', 'ministra', 's...","['sr', 'presidente', 'srª', 'ministro', 'srª',...","[(sr presidente srª ministra srª, 'PER', 431..."
92573,António Filipe,PCP,"Sr. Presidente, Srs. Membros do Governo, Srs. ...",darl13sl03n058.txt,sr presidente srs membros do governo srs deput...,"['sr', 'presidente', 'srs', 'membros', 'do...","['sr', 'presidente', 'sr', 'membro', 'do', 'go...","[(sr presidente srs, 'PER', 4317129024397789..."


In [4]:
#token frequency by party

def flatten(t):
    return [item for sublist in t for item in sublist]

counter_by_party = {}

for party in list(set(declaracoes_nopunct["partido"])):
    party_rows = declaracoes_nopunct.loc[declaracoes_nopunct["partido"] == party]
    party_tokens = flatten(party_rows["tokens"])
    #party_tokens = [token.lower() for token in party_tokens]
    party_tokens = [token.lstrip(" ") for token in party_tokens]
    counter = [Counter(party_tokens)]

    counter_by_party[party] = counter
    
counter_by_party

{'CH': [Counter({"'sr'": 1752,
           "'presidente'": 1069,
           "'ministro'": 268,
           "'da'": 1524,
           "'administração'": 41,
           "'interna'": 38,
           "'o'": 5167,
           "'tom'": 4,
           "'que'": 8309,
           "'imprimiu'": 1,
           "'ao'": 639,
           "'seu'": 221,
           "'discurso'": 12,
           "'hoje'": 634,
           "'nesta'": 187,
           "'assembleia'": 103,
           "'dizer'": 550,
           "'eu'": 163,
           "'tive'": 4,
           "'razão'": 81,
           "'governo'": 701,
           "'teve'": 44,
           "'tivemos'": 75,
           "'todos'": 436,
           "'e'": 4190,
           "'a'": 5403,
           "'falta'": 89,
           "'de'": 6235,
           "'entusiasmo'": 2,
           "'com'": 1083,
           "'partido'": 487,
           "'socialista'": 449,
           "'bateu'": 1,
           "'palmas'": 14,
           "'no'": 777,
           "'final'": 35,
           "'quase'": 40,
 

In [5]:
tokens_by_party = pd.concat({k: pd.DataFrame(v).T for k, v in counter_by_party.items()}, axis=0)
tokens_by_party = tokens_by_party.stack().unstack(level=1)
tokens_by_party = tokens_by_party.transpose()
tokens_by_party = tokens_by_party.fillna(0)
tokens_by_party = tokens_by_party.astype("int") 
tokens_by_party = tokens_by_party.reset_index()
tokens_by_party = tokens_by_party.droplevel(1, axis=1)
tokens_by_party["index"] = [token.replace("\'\'", "") for token in tokens_by_party["index"]]
#tokens_by_party.to_csv("tokens_by_party.csv", index=False)

In [6]:
def flatten(t):
    return [item for sublist in t for item in sublist]

counter_by_party = {}

for party in list(set(declaracoes_nopunct["partido"])):
    party_rows = declaracoes_nopunct.loc[declaracoes_nopunct["partido"] == party]
    party_entities = flatten(party_rows["entities"])
    #party_tokens = [token.lower() for token in party_tokens]
    party_entities = [entity.lstrip(" ") for entity in party_entities]
    counter = [Counter(party_entities)]

    counter_by_party[party] = counter
    
counter_by_party

{'CH': [Counter({'(sr presidente sr': 24,
           "'PER'": 1773,
           '4317129024397789502)': 1773,
           '(europa': 139,
           "'LOC'": 1310,
           '385)': 1310,
           '(sr ministro': 24,
           '(espanha': 22,
           '(sr': 76,
           '(srª ministra': 22,
           '(mamadou ba': 1,
           '(fabíola cardoso': 1,
           '(sr presidente': 268,
           '': 467,
           '(adão silva': 1,
           '(união europeia': 145,
           "'ORG'": 1021,
           '383)': 1021,
           '(em portugal': 181,
           '(sr presidente srs deputados': 78,
           '(sr deputado pedro do carmo': 2,
           '(venezuela': 14,
           '(cuba': 6,
           '(dr mário soares': 1,
           '(sr presidente sras': 34,
           '(sr deputado antónio filipe': 9,
           '(sr deputado duarte alves': 2,
           '(sr presidente srs': 62,
           '(sr primeiroministro': 180,
           '(tribunal de contas': 18,
           '(andré

In [7]:
entities_by_party = pd.concat({k: pd.DataFrame(v).T for k, v in counter_by_party.items()}, axis=0)
entities_by_party = entities_by_party.stack().unstack(level=1)
entities_by_party = entities_by_party.transpose()
entities_by_party = entities_by_party.fillna(0)
entities_by_party = entities_by_party.astype("int") 
entities_by_party = entities_by_party.reset_index()
entities_by_party = entities_by_party.droplevel(1, axis=1)
entities_by_party["index"] = [token.replace("\'\'", "") for token in entities_by_party["index"]]

In [8]:
entities_by_party.to_csv("entities.csv", index=False)

In [9]:
#declaracoes_nopunct.groupby("partido")["speaker"].count()
declaracoes_nopunct["speaker"].value_counts()#.to_csv("speakers.csv")

João Oliveira            4621
Nuno Magalhães           2183
Cecília Meireles         2043
Pedro Filipe Soares      2023
João Pinho de Almeida    1852
                         ... 
Pedro Filipe Sores          1
IsabelLopes                 1
Mário Centeno               1
Capoulas Santos             1
Catarina Marcelina          1
Name: speaker, Length: 834, dtype: int64

In [15]:
declaracoes_nopunct.groupby(by=["partido", "filename"])["speaker"].unique()#.to_csv("speakers_by_party.csv")

,speaker,partido,speech,filename,no_punct,tokens,lemma,entities
0,Emília Cerqueira,PSD,"Sr. Presidente, Srs. Deputados, saúdo Os Verde...",darl13sl03n050.txt,sr presidente srs deputados saúdo os verdes po...,"['sr', 'presidente', 'srs', 'deputados', '...","['sr', 'presidente', 'sr', 'deputar', 'saudar'...","[(sr presidente srs, 'PER', 4317129024397789..."
1,Emília Cerqueira,PSD,"Portanto, estamos a falar, infelizmente, de um...",darl13sl03n050.txt,portanto estamos a falar infelizmente de uma s...,"['portanto', 'estamos', 'a', 'falar', 'inf...","['portanto', 'estar', 'o', 'falar', 'infelizme...","[(espanha, 'LOC', 385)]"
2,Emília Cerqueira,PSD,"Terminarei, Sr. Presidente, perguntando a Os V...",darl13sl03n050.txt,terminarei sr presidente perguntando a os verd...,"['terminarei', 'sr', 'presidente', 'pergunt...","['terminar', 'sr', 'presidente', 'perguntar', ...","[(terminarei sr, 'PER', 4317129024397789502)..."
3,Emília Cerqueira,PSD,Não é verdade!,darl13sl03n050.txt,não é verdade,"['não', 'é', 'verdade']","['não', 'ser', 'verdade']",[]
4,Emília Cerqueira,PSD,"Sr. Presidente, Sr. Deputado Pedro Soares, hoj...",darl13sl03n050.txt,sr presidente sr deputado pedro soares hoje te...,"['sr', 'presidente', 'sr', 'deputado', 'pe...","['sr', 'presidente', 'sr', 'deputar', 'pedrar'...","[(sr presidente sr deputado pedro soares, 'PE..."
...,...,...,...,...,...,...,...,...
92570,Heloísa Apolónia,PEV,"Sr. Presidente, Sr.as e Srs. Deputados: O prob...",darl13sl03n058.txt,sr presidente sras e srs deputados o problema ...,"['sr', 'presidente', 'sras', 'e', 'srs', ...","['sr', 'presidente', 'sr', 'e', 'sr', 'deputar...","[(sr presidente sras, 'PER', 431712902439778..."
92571,Heloísa Apolónia,PEV,"É por estas razões e por tantas outras, Sr. Pr...",darl13sl03n058.txt,é por estas razões e por tantas outras sr pres...,"['é', 'por', 'estas', 'razões', 'e', 'por...","['ser', 'por', 'este', 'razão', 'e', 'por', 't...",[]
92572,António Filipe,PCP,"Sr. Presidente, Sr.ª Ministra, Sr.ª Secretária...",darl13sl03n058.txt,sr presidente srª ministra srª secretária de e...,"['sr', 'presidente', 'srª', 'ministra', 's...","['sr', 'presidente', 'srª', 'ministro', 'srª',...","[(sr presidente srª ministra srª, 'PER', 431..."
92573,António Filipe,PCP,"Sr. Presidente, Srs. Membros do Governo, Srs. ...",darl13sl03n058.txt,sr presidente srs membros do governo srs deput...,"['sr', 'presidente', 'srs', 'membros', 'do...","['sr', 'presidente', 'sr', 'membro', 'do', 'go...","[(sr presidente srs, 'PER', 4317129024397789..."


In [19]:
'''entities = declaracoes[["filename", "partido", "entities"]]

counter = {}
counter_party = {}

for party in list(set( entities["partido"] )):
    counter_party.setdefault(party, {})

    for ent_list in entities["entities"]:
        list_of_all_items  = str(ent_list).replace("[", "").replace("]", "").split(", ")
        grouped_items = np.array_split(list_of_all_items,3)
        stringified_groups = [", ".join(x) for x in grouped_items]
        
        for ent in stringified_groups:

            print("ent:")
            print((ent))
            # counter_party[party].setdefault(ent, 0)
            # counter_party[party][ent] += 1


    # counter_party = 1'''


'''for lst in entities["entities"]:
        for tup in lst:
          tup = tup.replace('(','')
          tup = tup.replace(')','')
          try:
            counter[tup] +=1
          except KeyError:
            counter[tup] = 1'''


#counter_party

'for lst in entities["entities"]:\n        for tup in lst:\n          tup = tup.replace(\'(\',\'\')\n          tup = tup.replace(\')\',\'\')\n          try:\n            counter[tup] +=1\n          except KeyError:\n            counter[tup] = 1'

In [ ]:
'''entities = declaracoes[["filename", "partido", "entities"]]

counter = {}
counter_party = 0

for party in entities["partido"]:
    counter['partido'] = party
    counter_party = +1

for fake_string in entities["entities"]:
    fake_string = fake_string.replace('[','')
    fake_string = fake_string.replace('(','')
    fake_string = fake_string.replace(')','')
    fake_string = fake_string.replace(']','')
    fake_string = fake_string.replace('\'','')
    fake_string = fake_string.replace('\"','')
    fake_string = fake_string.split(',')
    
    for name in fake_string:
        name = name.lstrip(" ")
        try:
            counter[name] +=1
        except KeyError:
            counter[name] = 1

counter'''
